# Data Exploration

This notebook is more or less a sandbox made for me to understand and decide how to prepare the dataset and what target to select.

### Constants and imports

In [1]:
import os
import pandas as pd
import pycountry
import pygwalker as pyg

METADATA_DIRECTORY = 'metadata'
JOINED_METADATA_FILE = os.path.join(METADATA_DIRECTORY, 'metadata_joined.csv')

### Loading common dataset

In [2]:
df = pd.read_csv(JOINED_METADATA_FILE)

df.head()

,id,title,date,begin_date,end_date,culture,technique,type,department,collection,path
0,cma_122190,Long Sampler,c. 1650–70,NaN,NaN,"England, 17th century",embroidery: silk on linen tabby ground,Sampler,Textiles,Textiles,dataset/raw/cma_122190.jpg
1,cma_171485,"Vanity Fair: Statesman, No. 57 ""A man of Fashi...",1870,NaN,NaN,"Italy, 19th century",color lithograph,Print,Prints,PR - Lithograph,dataset/raw/cma_171485.jpg
2,cma_161711,Cellarette,c. 1840,NaN,NaN,"America, New York",chiefly rosewood veneer with pine and poplar s...,Furniture and woodwork,Decorative Art and Design,Furniture,dataset/raw/cma_161711.jpg
3,cma_102749,Gradual,c. 1520,NaN,NaN,"Italy, Verona","Ink, tempera, and gold on vellum; wood binding",Manuscript,Medieval Art,MED - Manuscript Illuminations,dataset/raw/cma_102749.jpg
4,cma_101229,Fragment,1800s,NaN,NaN,"China, 19th century",silk,Textile,Textiles,Textiles,dataset/raw/cma_101229.jpg


### Checking type values

In [3]:
walker = pyg.walk(df)

Box(children=(HTML(value='<div id="ifr-pyg-0006100a5c150824UC7yWNp2dLYlKbBR" style="height: auto">\n    <head>…

### Extracting culture values

In [ ]:
country_names = [x.name for x in list(pycountry.countries)] + [x.name for x in list(pycountry.historic_countries)]

map_cache = {}

def mapping(x):
    if x in map_cache:
        return map_cache[x]
    try:
        result = pycountry.countries.search_fuzzy(x)[0]
        if 'country' in result:
            result = result.country.name
        else:
            result = result.name
        map_cache[x] = result
        return result
    except:
        try:
            result = pycountry.historic_countries.search_fuzzy(x)[0]
            if 'country' in result:
                result = result.country.name
            else:
                result = result.name
            map_cache[x] = result
            return result
        except:
            y = x.split(',')[0]
            if y in map_cache:
                map_cache[x] = map_cache[y]
                return map_cache[y]
            try:
                result = pycountry.countries.search_fuzzy(y)[0]
                if 'country' in result:
                    result = result.country.name
                else:
                    result = result.name
                map_cache[x] = result
                return result
            except:
                try:
                    result = pycountry.historic_countries.search_fuzzy(y)[0]
                    if 'country' in result:
                        result = result.country.name
                    else:
                        result = result.name
                    map_cache[x] = result
                    return result
                except:
                    map_cache[x] = None
                    return None
    map_cache[x] = None
    return None

df['country'] = df['culture'].map(mapping)

SubdivisionHierarchy(code='GB-ENG', country_code='GB', name='England', parent_code=None, type='Country')
SubdivisionHierarchy(code='GB-ENG', country_code='GB', name='England', parent_code=None, type='Country')
SubdivisionHierarchy(code='GB-ENG', country_code='GB', name='England', parent_code=None, type='Country')
SubdivisionHierarchy(code='MX-MEX', country_code='MX', name='México', parent_code=None, type='State')
SubdivisionHierarchy(code='MX-MEX', country_code='MX', name='México', parent_code=None, type='State')
SubdivisionHierarchy(code='US-CA', country_code='US', name='California', parent_code=None, type='State')
SubdivisionHierarchy(code='MX-MEX', country_code='MX', name='México', parent_code=None, type='State')
SubdivisionHierarchy(code='US-CA', country_code='US', name='California', parent_code=None, type='State')
SubdivisionHierarchy(code='MX-MEX', country_code='MX', name='México', parent_code=None, type='State')
SubdivisionHierarchy(code='MX-MEX', country_code='MX', name='México

In [ ]:
df['country']

### Preparing the final dataframe and dividing into train/test data

### Saving the .csv file